In [16]:
import os
import json
import pickle
import requests
import collections
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
with open('data/jeopardy_358974_questions_20190612.pkl', 'rb') as f:
    karl_to_question_id = pickle.load(f).to_dict()['questionid']
    
def get_result(card):
    record_id = karl_to_question_id[int(card['question_id'])]
    prob = (records_df[records_df.question_id == record_id]['correct'] / 2 + 0.5).mean()
    return 'correct' if prob > 0.5 else 'wrong'

In [ ]:
param = {
    'qrep': 0.7,
    'prob_difficult': 0.3,
    'prob_easy': 10,
    'category': -0.7,
    'recency': 1.0,
    'leitner': 1.0,
    'lr_prob_correct': 0.1,
    'lr_prob_wrong': 0.05,
    'lr_qrep': 0.3,
}
r = requests.post('http://127.0.0.1:8000/api/karl/set_hyperparameter', data=json.dumps(param))
r = json.loads(r.text)

with open('diagnostic_flashcards.pkl', 'rb') as f:
    flashcards = pickle.load(f)
    
requests.post('http://127.0.0.1:8000/api/karl/reset')

shown_cards = []
while len(shown_cards) < 300:
    r = requests.post('http://127.0.0.1:8000/api/karl/schedule', data=json.dumps(flashcards))
    r = json.loads(r.text)
    card_order = r['card_order']
    # print(r['curr_prob'], r['probs'][card_order[0]])
    show_card = flashcards[card_order[0]]
    show_card['label'] = get_result(show_card)
    print('{: <40} {: <20} {: <20}'.format(show_card['answer'], show_card['category'], show_card['label']))
    shown_cards.append(flashcards[card_order[0]])
    r = requests.post('http://127.0.0.1:8000/api/karl/update', data=json.dumps([show_card]))
    flashcards = [x for x in flashcards if x['question_id'] != show_card['question_id']]

In [2]:
with open('data/jeopardy_358974_questions_20190612.pkl', 'rb') as f:
    roger_questions = pickle.load(f)
with open('data/jeopardy_310326_question_player_pairs_20190612.pkl', 'rb') as f:
    roger_records = pickle.load(f)
with open('data/searchqa.json', 'r') as f:
    searchqa_questions = json.load(f)

In [3]:
len(searchqa_questions)

216930

In [4]:
roger_records

,question_id,player_id,correct,timestamp
73,697-2-4,player_007,1,2010-01-01 00:00:00
74,697-2-5,player_007,1,2010-01-01 00:00:00
75,697-3-1,player_007,1,2010-01-01 00:00:00
76,697-3-2,player_007,1,2010-01-01 00:00:00
77,697-3-3,player_007,1,2010-01-01 00:00:00
...,...,...,...,...
319112,2418-13-1,player_005,1,2019-06-06 23:59:22
319113,3620-13-1,player_005,1,2019-06-06 23:59:32
319114,1266-13-1,player_005,-1,2019-06-06 23:59:43
319115,3373-13-1,player_005,1,2019-06-06 23:59:50


In [5]:
searchqa_questions[0]

{'category': 'HISTORY',
 'air_date': '2004-12-31',
 'question': "'For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory'",
 'value': '$200',
 'answer': 'Copernicus',
 'round': 'Jeopardy!',
 'show_number': '4680'}

In [19]:
from whoosh.index import create_in
from whoosh.fields import *

schema = Schema(
    category=KEYWORD(stored=True),
    air_date=DATETIME(stored=True),
    question=TEXT(stored=True),
    value=STORED,
    answer=KEYWORD(stored=True),
    round=KEYWORD(stored=True),
    show_number=KEYWORD(stored=True),
)
index_dir = 'whoosh_index'
if not os.path.exists(index_dir):
    os.mkdir(index_dir)
ix = create_in(index_dir, schema)
writer = ix.writer()
for q in tqdm(searchqa_questions):
    writer.add_document(
        category=q['category'],
        air_date=q['air_date'],
        question=q['question'],
        value=q['value'],
        answer=q['answer'],
        round=q['round'],
        show_number=q['show_number']
    )
writer.commit()

100%|██████████| 216930/216930 [03:09<00:00, 1142.34it/s]


In [20]:
from whoosh.qparser import QueryParser
with ix.searcher() as searcher:
    query = QueryParser("question", ix.schema).parse("his life")
    results = searcher.search(query)
    print(results[0])

<Hit {'air_date': '2008-02-05', 'answer': 'Napoleon', 'category': 'NONFICTION PEOPLE', 'question': '\'"The Corsican" is a diary of his life "In His Own Words"\'', 'round': 'Jeopardy!', 'show_number': '5392', 'value': '$1000'}>
